# Alignment Error Visualization

This notebook collects COM data from the database and tries to quantify some alignment errors. The main results are shown in the plots at the end of the notebook.

In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from collections import OrderedDict
from IPython.display import HTML
from itertools import combinations
from scipy.ndimage import affine_transform
import SimpleITK as sitk


PIPELINE_ROOT = Path('./').absolute().parents[1]
PIPELINE_ROOT = PIPELINE_ROOT.as_posix()
sys.path.append(PIPELINE_ROOT)
print(PIPELINE_ROOT)


/home/eddyod/programming/pipeline/src


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from library.controller.sql_controller import SqlController
from library.image_manipulation.filelocation_manager import FileLocationManager
from library.atlas.atlas_utilities import apply_affine_transform, get_affine_transformation, \
fetch_coms, list_coms, compute_affine_transformation
from library.atlas.brain_structure_manager import BrainStructureManager
from library.utilities.utilities_process import M_UM_SCALE, SCALING_FACTOR, random_string, \
read_image, write_image


In [4]:
def sum_square_com(com):
    ss = np.sqrt(sum([s**2 for s in com]))
    return ss

def apply_affine_transformation(volume, matrix):
    """Apply an affine transformation to a 3D volume."""
    transformed_volume = affine_transform(volume, matrix, offset=0, order=1)
    return transformed_volume

In [5]:
moving_name = 'DK55'
fixed_name = 'AtlasV8'
moving_all = list_coms(moving_name, scaling_factor=1)
fixed_all = fetch_coms(fixed_name, scaling_factor=1)
common_keys = list(moving_all.keys() & fixed_all.keys())
bad_keys = ('RtTg', 'AP')
#bad_keys = ('RtTg',)
bad_keys = ()
good_keys = set(common_keys) - set(bad_keys)

moving_src = np.array([moving_all[s] for s in good_keys])
fixed_src = np.array([fixed_all[s] for s in good_keys])
print(len(common_keys))

29


In [6]:
moving_all

{'7N_L': [13080.7, 7323.6, 3660.0],
 '7N_R': [12927.3, 7183.8, 6300.0],
 'Tz_R': [12015.9, 7570.06, 5480.0],
 'Tz_L': [12009.4, 7635.5, 4460.0],
 '5N_R': [12198.9, 6011.09, 6320.0],
 'Pn_L': [10897.3, 7409.56, 4280.0],
 'Pn_R': [11083.8, 7421.77, 5580.0],
 '3N_L': [11207.5, 5005.76, 4620.0],
 '3N_R': [11172.8, 4960.67, 4900.0],
 '6N_R': [12691.9, 6174.66, 5240.0],
 '6N_L': [12747.7, 6215.66, 4500.0],
 '10N_R': [14330.9, 5936.56, 5240.0],
 'Amb_R': [13785.2, 6869.11, 6340.0],
 'Amb_L': [13717.5, 7135.54, 3660.0],
 'DC_L': [13327.7, 5646.89, 2660.0],
 'DC_R': [13334.3, 5283.99, 7080.0],
 'LC_R': [12679.9, 5218.19, 5680.0],
 'LC_L': [12584.0, 5309.77, 3960.0],
 'LRt_L': [14545.3, 7358.92, 3680.0],
 'LRt_R': [14848.1, 7225.04, 6280.0],
 '5N_L': [12184.6, 6245.08, 3400.0],
 'PBG_R': [11309.0, 4916.5, 6740.0],
 'PBG_L': [11314.1, 5133.68, 2860.0],
 'AP': [14439.6, 5705.57, 4920.0],
 '4N_L': [11525.2, 5043.96, 4460.0],
 '4N_R': [11474.0, 4986.83, 4940.0],
 '10N_L': [14451.13, 6652.42, 5820.0]

In [7]:
#transformation_matrix = get_affine_transformation(moving_name=moving_name, fixed_name=fixed_name, 
#                                                  scaling_factor=1)
transformation_matrix = compute_affine_transformation(moving_src, fixed_src)
print(transformation_matrix)

[[ 1.00161836e+00 -3.60715444e-03 -1.09577172e-02 -1.12492477e+02]
 [ 7.95601847e-02  7.99077250e-01 -1.18628904e-02  4.38934248e+02]
 [-1.06040005e-01 -2.72150865e-02  9.07737472e-01  1.47314659e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [8]:
df_list = []
error = []
transformed_dict = {}
for structure in common_keys:
    moving0 = np.array(moving_all[structure])
    fixed0 = np.array(fixed_all[structure]) 
    transformed = apply_affine_transform(moving0, transformation_matrix)
    transformed = [x for x in transformed]
    difference = [a - b for a, b in zip(transformed, fixed0)]
    ss = sum_square_com(difference)
    row = [structure, np.round(moving0), np.round(fixed0), 
           np.round(transformed), np.round(difference), ss]
    df_list.append(row)
    error.append(ss)
    transformed_dict[structure] = transformed
print('RMS', sum(error)/len(df_list))
# MD589 to Allen RMS 260.0211852431133
# MD585 to Allen RMS 263.314352291951
# MD594 to Allen RMS 250.79820210419254
# AtlasV8 disk to Allen RMS 237.0680595008574
# MD585 to MD589 RMS 18.2658167690059

RMS 190.01406664601984


In [9]:
#transformation_matrix = np.hstack([transformation_matrix, t])
#transformation_matrix = np.vstack([transformation_matrix, np.array([0, 0, 0, 1])])
#print(transformation_matrix)
structure = 'SC'
try:
    com = moving_all[structure]
except KeyError:
    structure = common_keys[0]
    com = moving_all[structure]
#com = [1095, 392, 519]
print(f'{moving_name} {structure} non trans {np.round(np.array(com))}')
transformed_structure = apply_affine_transform(com, transformation_matrix)
print(f'{moving_name} {structure} apply trans {np.round(transformed_structure/1)}')
print(f'{fixed_name} {structure} {np.round(np.array(fixed_all[structure]))}')
diff = transformed_structure - fixed_all[structure]
print(f'{moving_name}->{fixed_name} {structure} {np.round(diff)}')


DK55 SC non trans [11190.  3080.  4640.]
DK55 SC apply trans [11034.  3735.  4415.]
AtlasV8 SC [10949.  3924.  4399.]
DK55->AtlasV8 SC [  85. -188.   16.]


In [10]:
columns = ['structure', moving_name, fixed_name, 'transformed', 'difference', 'sumsquares']
df = pd.DataFrame(df_list, columns=columns)
df.index.name = 'Index'
df = df.round(4)
df.sort_values(by=['sumsquares'], inplace=True)
#df.to_csv('/home/eddyod/programming/pipeline/docs/sphinx/source/_static/results.csv', index=False)
df.head(50)

,structure,DK55,AtlasV8,transformed,difference,sumsquares
Index,,,,,,
26,3N_L,"[11208.0, 5006.0, 4620.0]","[11014.0, 5264.0, 4317.0]","[11044.0, 5276.0, 4342.0]","[30.0, 12.0, 25.0]",41.1264
23,Tz_L,"[12009.0, 7636.0, 4460.0]","[11831.0, 7413.0, 4009.0]","[11840.0, 7443.0, 4040.0]","[9.0, 30.0, 31.0]",44.0398
4,Tz_R,"[12016.0, 7570.0, 5480.0]","[11818.0, 7395.0, 5017.0]","[11835.0, 7379.0, 4967.0]","[17.0, -16.0, -50.0]",54.9590
25,5N_L,"[12185.0, 6245.0, 3400.0]","[12014.0, 6306.0, 3088.0]","[12032.0, 6358.0, 3097.0]","[18.0, 52.0, 9.0]",55.9804
6,3N_R,"[11173.0, 4961.0, 4900.0]","[10985.0, 5279.0, 4575.0]","[11007.0, 5234.0, 4601.0]","[22.0, -45.0, 27.0]",56.6016
17,4N_L,"[11525.0, 5044.0, 4460.0]","[11365.0, 5279.0, 4179.0]","[11364.0, 5333.0, 4162.0]","[-0.0, 55.0, -16.0]",57.2045
10,4N_R,"[11474.0, 4987.0, 4940.0]","[11337.0, 5280.0, 4674.0]","[11308.0, 5278.0, 4605.0]","[-29.0, -2.0, -69.0]",74.8904
21,5N_R,"[12199.0, 6011.0, 6320.0]","[11980.0, 6184.0, 5809.0]","[12015.0, 6138.0, 5753.0]","[35.0, -47.0, -56.0]",80.8483
5,6N_L,"[12748.0, 6216.0, 4500.0]","[12584.0, 6276.0, 4057.0]","[12584.0, 6367.0, 4037.0]","[-0.0, 91.0, -20.0]",92.6573
